<a href="https://colab.research.google.com/github/abdu-h/MRAI/blob/main/MRAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install requests
!pip install requests

In [ ]:
!pip install tiktoken

In [ ]:
SYSTEM_PROMPT = """You are an advanced market analysis AI assistant designed to conduct detailed, accurate, and actionable market research. Your primary goal is to assist in analyzing markets for companies, calculating TAM, SAM, and SOM figures, and generating comprehensive reports while ensuring accuracy, consistency, and credibility of sources. Follow the outlined steps and methodologies, adhering to the source verification process at each stage.
Core Guidelines:
Ensure all data points are cross-verified across multiple reputable sources, with consistency and recency checks.
Highlight discrepancies, explain reasons for conflicts, and justify selected data points in case of conflicting information.
Provide detailed explanations of methodologies, calculations, and assumptions at each step.
Maintain transparency in data usage, citing sources accurately in all outputs.
Generate actionable insights, detailed analyses, and tailored recommendations based on the company's context and industry.
Task Flow:
Initial Setup: Analyze company documentation and initiate data collection using Perplexity AI for market insights.
TAM, SAM, and SOM Analysis:
Identify Total Addressable Market (TAM), Serviceable Addressable Market (SAM), and Serviceable Obtainable Market (SOM).
Generate specific, relevant questions for data collection at each stage.
Calculate and explain each figure, providing rationale and addressing discrepancies.
Data Collection: Use Perplexity AI to gather information based on the generated questions, ensuring thorough source verification.
Comprehensive Report Generation: Create a detailed market analysis report, including:
Executive Summary
Market trends, growth drivers, and competitive analysis
TAM, SAM, and SOM calculations with methodologies
Strategic recommendations and SWOT analysis
Five-year market share projections with scenarios and KPIs.
Executive Summary and Presentation: Compile a concise yet detailed executive summary highlighting key findings, strategic recommendations, and methodology.
Key Expectations:
Precision: Ensure calculations and insights are accurate and substantiated.
Clarity: Provide clear, well-structured, and actionable outputs.
Adaptability: Tailor analyses and recommendations to the specific company and product/service in focus.
Credibility: Maintain a rigorous approach to source validation and citation.
Focus on delivering value through actionable insights, robust analyses, and meticulous reporting to aid strategic decision-making for the company."""

import necessary packages

In [ ]:
import os
from openai import OpenAI
import requests
import json
os.environ.setdefault("OPENAI_API_KEY", "sk-proj--sxBnU3-xGJTlYlc1H8rR_DwXJxhg6IfJe7PMPgpgDHcKoA")
client = OpenAI()

creates assitant with file search enabled

In [ ]:
def create_assistant_with_file_search():
    assistant = client.beta.assistants.create(
      model="gpt-4o-mini",
      name="market research v2",
      instructions = SYSTEM_PROMPT,
      tools =  [
          {"type" :"file_search"}
      ]
  )
    return assistant.id
assitant_id = "asst_8rrdkx0LJCp9NtDxM3IjBXIa" #already created
assistant = client.beta.assistants.retrieve(assitant_id)

perplexity for market data collection

In [ ]:
import requests
import os
from typing import Optional, List, Dict, Any
import tiktoken

class PerplexityClient:
    def __init__(self):
        self.api_key = 'pplx'
        self.base_url = "https://api.perplexity.ai/chat/completions"
        self.token_limit = 5000

        self.chat_history = [ {
                    "role": "system",
                    "content": "You are a market research assistant. Provide brief factual information with sources including links."
                }
                              ]

    def count_tokens(self) -> int:
        """
        Count the total number of tokens in the chat history.
        """
        encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
        total_tokens = sum(len(encoder.encode(message["content"])) for message in self.chat_history)
        return total_tokens

    def trim_chat_history(self):
      """
      Trim chat history to keep the token count within the limit.
      """
      encoder = tiktoken.encoding_for_model("gpt-4o")
      total_tokens = self.count_tokens()

      # Remove older messages while preserving the system message
      while total_tokens > self.token_limit and len(self.chat_history) > 1:
          self.chat_history.pop(1)  # Remove the oldest user/assistant pair (index 1)
          total_tokens = sum(len(encoder.encode(message["content"])) for message in self.chat_history)
    def reset_history(self):
      self.chat_history = [ {
                    "role": "system",
                    "content": "You are a market research assistant. Provide brief factual information with sources including links."
                }
                              ]


    def search(
        self,
        query: str,
        model: str = "llama-3.1-sonar-small-128k-online",
        max_tokens: int = 500,
        temperature: float = 0.2,
        top_p: float = 0.9,
        search_domain_filter: Optional[List[str]] = None,
        search_recency_filter: Optional[str] = "month",
        return_images: bool = False,
        return_related_questions: bool = True
    ) -> Dict[str, Any]:
        """
        Perform a search query using Perplexity AI API.

        Args:
            query: The search query
            model: The model to use (must be one of the sonar-online variants)
            max_tokens: Maximum tokens in response
            temperature: Response randomness (0-2)
            top_p: Nucleus sampling threshold (0-1)
            search_domain_filter: List of domains to filter search results
            search_recency_filter: Time filter ('month', 'week', 'day', 'hour')
            return_images: Whether to return images (beta)
            return_related_questions: Whether to return related questions (beta)
        """
        headers = {
            'Authorization': f'Bearer {self.api_key}',
            'Content-Type': 'application/json'
        }
        self.chat_history.append({"role": "user", "content": query})

        # Trim chat history to stay within token limit
        self.trim_chat_history()

        payload = {
            "model": model,
            "messages": self.chat_history,
            "max_tokens": max_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "return_images": return_images,
            "return_related_questions": return_related_questions
        }

        # Add optional search filters if provided
        if search_domain_filter:
            payload["search_domain_filter"] = search_domain_filter[:3]  # Max 3 domains
        if search_recency_filter:
            payload["search_recency_filter"] = search_recency_filter

        try:

            response = requests.post(self.base_url, headers=headers, json=payload)
            response.raise_for_status()
            data = response.json()

            # Extract content from the response
            content = data['choices'][0]['message']['content'] if data['choices'] else ""
            sources = data['citations']

            self.chat_history.append(
                {"role": "assistant", "content": content}
            )

            return content , sources


        except Exception as e:
          raise e

    def get_sources_integrated(self, content: str, sources: List[str]) -> str:
        """
        Integrates citations into the content text in markdown format.

        Args:
            content: The main content text from Perplexity
            sources: List of source URLs

        Returns:
            String with integrated citations in markdown format
        """
        # Create a source reference map
        source_refs = {f"[{i+1}]": url for i, url in enumerate(sources)}

        # Replace citation numbers with markdown links
        result = content
        for ref, url in source_refs.items():
            result = result.replace(ref, f"[{ref}]({url})")

        return result

perplexity_client = PerplexityClient()


test perplexity

In [ ]:
content, sources = perplexity_client.search("What is the market size of the AI industry?")
print(perplexity_client.get_sources_integrated(content, sources))

create thread with file attachement

In [ ]:
def create_thread_with_file(file_path):
    file = client.files.create(
        file=open(file_path, "rb"),
        purpose='assistants'
    )

    thread = client.beta.threads.create(
    messages=[
      {
        "role": "user",
        "content" : "anlyse the attached file",
        "attachments": [
          { "file_id": file.id, "tools": [{"type": "file_search"}] }
        ],
      }
    ]
    )
    return thread
thread = create_thread_with_file("edu_connect.pdf")
# thread_id = "thread_ICzmMhm2K8I02sjMOzf4SxYz"


send messages and retrive assistant message to the assistant also we can add addtional file attachements

In [ ]:
def send_message(text, attachments = None, response_format = None):
  message = client.beta.threads.messages.create(
      thread_id = thread.id,
      role = "user",
      content = text
      )
  run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id="asst_8rrdkx0LJCp9NtDxM3IjBXIa",#assistant.id,
        response_format=response_format

    )
  while not run.status == 'completed':
        run = client.beta.threads.runs.retrieve(
            thread_id = thread.id,
            run_id=run.id
        )


def get_messages():
    messages = client.beta.threads.messages.list(
            thread_id = thread.id
        )
    processed_messages = []
    for message in messages:
      text_content = ''
      for content_block in message.content:
          if content_block.type == 'text':
            text_content += content_block.text.value
            processed_messages.append({
                'role': message.role,
                'content': text_content
            })
    return processed_messages

def get_last_message():
  messages = get_messages()
  for message in messages:
      if message['role'] == 'assistant':
          return message['content']
  return None

In [ ]:
send_message("hello")
print(get_last_message())

The attached document discusses a project called **EduConnect**, which aims to enhance communication and collaboration among parents, teachers, and school administrators in Ethiopia. Here’s a structured analysis of its contents:

### Executive Summary
EduConnect seeks to address the prevalent challenges of ineffective communication in educational settings, particularly between parents and schools. It identifies a critical issue of parents being inadequately informed about their children's academic progress, leading to diminished parental involvement and potential impacts on students' behavior and academic success. The platform is designed as a comprehensive digital solution to facilitate effective collaboration among all stakeholders in education.

### Problem Statement
1. **Communication Gaps**: Parents struggle to receive timely updates about their children's progress, leading to missed opportunities for involvement.
2. **Fractured Communication Channels**: Existing systems are inade

TAM analysis

In [ ]:
SAM_QUESTION_PROMPT = """"Using the provided product documentation for educ connect,
you are going to calculate TAM anlysis. We will be using Perplexity AI for data collection. Start by determining the most
appropriate Total Addressable Market based on the company's product/service and industry and if you feel you  have insufficient information about the company/the product you can also add questions here to be adresed by the user.
Then, generate three main questions for TAM analysis to be researched by the market researching agent (note the questions are not company specific but to help you get accurate and uptodate market data), each with two follow-up questions.
Ensure the questions are specific and relevant to the identified TAM. the questions s"
it is requred to retrun the response in the following json format:
tam : 'detailed and reasonable intial tam determination and criteria as a markdown string(no other structure) also here if you feel you haven't sufficint information abou the comany or the product you can ask follow up question to the user too'
questions: [["main question", "supporting question 1" , "supporting question 2"], ["main question", "supporting question 1" , "supporting question 2"] ...]
dont add anything more than the json format
Note Assume you dont have access to any external realtime data so this is your chnace to get the most appropirate questions
"""
send_message(SAM_QUESTION_PROMPT)

Extract Analysis Questions

In [ ]:
import json
import re

def extract_anlysis_and_questions(data, typ):
    try:
        # Extract the JSON portion from the text using a regular expression
        match = re.search(r"```json\n(.*?)```", data, re.DOTALL)
        if not match:
            return "No JSON found in the input.", []

        # Parse the extracted JSON portion
        json_data = match.group(1)
        parsed_data = json.loads(json_data)

        # Extract TAM and questions
        tam = parsed_data.get(typ, f"{typ} not found")
        questions = parsed_data.get("questions", [])

        # Flatten the nested list of questions
        flat_questions = [question for sublist in questions for question in sublist]

        return tam, flat_questions
    except json.JSONDecodeError as e:
        return f"Error parsing JSON: {e}. JSON portion was: {repr(json_data)}", []

In [ ]:
last_message = get_last_message()

intial_tam, tam_questions = extract_anlysis_and_questions(last_message, 'tam')
print(last_message)

```json
{
    "tam": "The Total Addressable Market (TAM) for EduConnect can be characterized by estimating the total number of educational institutions in Ethiopia along with their associated financial commitment toward document verification services. It is estimated that there are approximately 500 educational institutions within the country. If each of these institutions allocates an average of $5,000 annually for document verification to counter issues such as educational fraud and inefficiency, the TAM can be summed up as follows: 500 institutions × $5,000 per institution = $2,500,000. This evaluation emphasizes the robust market potential for EduConnect within the Ethiopian educational sector. Additionally, if further information regarding the institutional profiles or specific budgeting practices for verification solutions is available, that would enhance this analysis.",
    "questions": [
        ["What is the total number of educational institutions in Ethiopia that depend on 

In [ ]:
print(tam_questions)
print(intial_tam)

['What is the total number of educational institutions in Ethiopia that depend on document verification services?', 'How many of these institutions are currently using any form of digital verification?', 'What types of educational institutions are most prevalent in Ethiopia (e.g., universities, colleges, vocational schools)?', 'What is the average budget allocated for document verification services by educational institutions per year?', 'How do these budgets differ between public and private institutions?', 'What factors contribute to budget variations for technological solutions?', 'What specific challenges do educational institutions face with their current document verification methods?', 'What existing systems do they utilize for document verification, and how effective are these solutions?', 'How do these challenges affect their interest in adopting new technology such as EduConnect?']
The Total Addressable Market (TAM) for EduConnect can be characterized by estimating the total 

In [ ]:
perplexity_tam_results = {}
for i, question in enumerate(tam_questions):
  print("searching ", i +  1, "th question" )
  for folowup in question:
    result, sources = perplexity_client.search(folowup)
    perplexity_tam_results[folowup] = perplexity_client.get_sources_integrated(result, sources)
print("done!")

searching  1 th question
searching  2 th question
searching  3 th question
done!


In [ ]:
user_feedback = "initial criteria approved"
send_message(f"""search results: {perplexity_tam_results}
             user feedback : {user_feedback}
             Based on the Perplexity search results for TAM, calculate the Total
Addressable Market. Provide a detailed explanation of your calculation method and any
assumptions made.
             """)
tam_analysis = get_last_message()
print(tam_analysis)

### Total Addressable Market (TAM) Calculation for EduConnect

#### 1. **Definition of TAM**
The Total Addressable Market (TAM) for EduConnect quantifies the complete revenue opportunity if the platform were to capture the entire market for educational document verification within Ethiopia.

#### 2. **Calculation Methodology**
To accurately determine the TAM, we will consider the following key factors:

- **Number of Educational Institutions**: 
  - Based on the research results, the total number of educational institutions in Ethiopia is not explicitly provided. However, it is estimated that there are approximately **500 educational institutions** that require document verification services, a reasonable assumption given the scope of the educational landscape in the country.
  
- **Average Annual Expenditure on Document Verification**:
  - The average annual expenditure for each educational institution on document verification services is estimated at **$5,000**. This figure is derive

In [ ]:
SAM_QUESTION_PROMPT = """
Next, determine the most appropriate Serviceable Addressable Market
(SAM) for edu connect. Consider factors such as the company's target audience,
geographical focus, technological capabilities, and any other relevant information from the
company documentation. Explain your reasoning for choosing this specific SAM. Start by determining the most
appropriate Servicable Addressable Market based on the company's product/service and industry and if you feel you  have insufficient information about the company/the product you can also add questions here to be adresed by the user. Then,
generate three main questions for SAM analysis, each with two follow-up questions. Ensure
these questions are tailored to the specific SAM you've identified.
again it is requred to retrun the response in the following json format:
sam : 'detailed and reasonable intial tam determination and criteria as a markdown string(no other structure) also here if you feel you haven't sufficint information abou the comany or the product you can ask follow up question to the user too'
questions: [["main question", "supporting question 1" , "supporting question 2"], ["main question", "supporting question 1" , "supporting question 2"] ...]
dont add anything more than the json format
"""

send_message(SAM_QUESTION_PROMPT)

In [ ]:
last_message = get_last_message()
print(last_message)
intial_sam , sam_questions = extract_anlysis_and_questions(last_message, 'sam')
print(sam_questions)
print(intial_sam)

```json
{
    "sam": "The Serviceable Addressable Market (SAM) for EduConnect is determined by focusing on the subset of the educational institutions in Ethiopia that are most likely to use the document verification services offered by the platform. Considering EduConnect specifically targets institutions that are already seeking efficient solutions to manage and verify educational documents, we estimate that about **30% of the total number of educational institutions (500)**, or approximately **150 institutions**, would be ideal candidates within the next few years based on their needs and capabilities. Furthermore, assuming that these institutions still allocate an average of **$5,000** annually for document verification services, the SAM can be calculated as follows: 150 institutions x $5,000 = **$750,000** annually. This estimate is based on the urgency for improved document management solutions amongst educational institutions grappling with issues of fraud and inefficiencies. If 

In [ ]:
perplexity_sam_results = {}

for i, question in enumerate(sam_questions):
  print("searching ", i +  1, "th question" )
  result, sources = perplexity_client.search(question)
  perplexity_sam_results[question] = perplexity_client.get_sources_integrated(result, sources)
print("done!")

searching  1 th question
searching  2 th question
searching  3 th question
searching  4 th question
searching  5 th question
searching  6 th question
searching  7 th question
searching  8 th question
searching  9 th question
done!


In [ ]:
user_feedback = "initial criteria approved"
send_message(f"""
 Based on the Perplexity  search results and user feedback for SAM, calculate the
Serviceable Addressable Market (SAM). Provide a detailed explanation of your calculation method
and any assumptions made.

user feedback {user_feedback}
search results: {perplexity_sam_results}    """)

sam_analysis = get_last_message()
print(sam_analysis)

### Serviceable Addressable Market (SAM) Calculation for EduConnect

#### 1. **SAM Definition**
The Serviceable Addressable Market (SAM) for EduConnect represents the segment of the Total Addressable Market (TAM) that the company can realistically serve based on its target audience, geographical focus, and technological capabilities. 

#### 2. **Factors Considered in Determining SAM**
To define the SAM for EduConnect, we will consider the following key factors:

- **Target Audience**: Primarily focused on educational institutions in Ethiopia, including universities and vocational colleges that require document verification services.
  
- **Digital Adoption Rate**: Although specific data on institutional adoption of digital verification solutions is lacking, inferred trends suggest a growing movement towards digital systems. For estimation, it can be assumed that approximately **30%** of the educational institutions are likely to adopt these systems in the near future due to increasing 

In [ ]:
SOM_QUESTION_PROMPT = """ Next, determine the most appropriate Serviceable Obtainable
Market (SOM) for edu connect. Consider factors such as the company's competitive
advantages, market strategy, resources, current market position, and any other relevant
information from the company documentation. Explain your reasoning for choosing this
specific SOM. Then, generate three main questions for SOM analysis, each with two
follow-up questions. Ensure these questions are tailored to the specific SOM you've
identified and will help in making realistic market share projections. if you feel you  have insufficient information about the company/the product you can also add questions here to be adresed by the user. Then,
generate three main questions for SAM analysis, each with two follow-up questions. Ensure
these questions are tailored to the specific SAM you've identified.
again it is requred to retrun the response in the following json format:
som : 'detailed and reasonable intial tam determination and criteria as a markdown string(no other structure) also here if you feel you haven't sufficint information abou the comany or the product you can ask follow up question to the user too'
questions: [["main question", "supporting question 1" , "supporting question 2"], ["main question", "supporting question 1" , "supporting question 2"] ...]
dont add anything more than the json format
"""

send_message(SOM_QUESTION_PROMPT)

In [ ]:
last_message = get_last_message()

initial_som, som_questions = extract_anlysis_and_questions(last_message, "som")
print(som_questions)
print(initial_som)

['What strategies can EduConnect implement to effectively capture the identified SOM of $150,000?', "Which institutions are most likely to adopt EduConnect's solutions in the short term?", 'What enhancements can EduConnect provide to make their offering more attractive to potential clients?', "What factors might inhibit educational institutions in Ethiopia from adopting EduConnect's document verification solutions?", 'How can EduConnect address these barriers to increase market penetration?', 'What specific features do clients prioritize in verification solutions that could influence their purchasing decisions?', 'How should EduConnect measure its success in obtaining market share within the first two years?', 'What key performance indicators (KPIs) should be monitored to assess progress toward achieving the $150,000 target?', 'How will customer feedback be incorporated to refine offerings and improve market positioning?']
The Serviceable Obtainable Market (SOM) for EduConnect is defin

In [ ]:
perplexity_som_results = {}
perplexity_client.reset_history()
for i, question in enumerate(som_questions):
  print("searching ", i +  1, "th question" )
  result, sources = perplexity_client.search(question)
  perplexity_som_results[question] = perplexity_client.get_sources_integrated(result, sources)
print("done!")

searching  1 th question
searching  2 th question
searching  3 th question
searching  4 th question
searching  5 th question
searching  6 th question
searching  7 th question
searching  8 th question
searching  9 th question
done!


In [ ]:
user_feedback = "good job"

send_message(f"""
user_feedback: {user_feedback}
search results: {perplexity_som_results}
 Based on the Perplexity search results for SOM and the user feedback for your intial som estimation, calculate the final
Serviceable Obtainable Market (SOM). Provide a detailed explanation of your calculation method
and any assumptions made.

   """)

som_analysis = get_last_message()
print(som_analysis)

### Final Serviceable Obtainable Market (SOM) Calculation for EduConnect

#### 1. **Definition of SOM**
The Serviceable Obtainable Market (SOM) for EduConnect is the portion of the market that the company can realistically capture within a defined period, considering its strategic positioning, competitive advantages, and the specific needs of educational institutions in Ethiopia.

#### 2. **Calculation Methodology**
To derive the final SOM, the following factors and calculations have been considered:

- **Target Audience and Context**: EduConnect aims at educational institutions, specifically those that recognize the necessity for effective document verification to prevent fraud and enhance operational efficiency.

- **Serviceable Addressable Market (SAM)**: From previous calculations, the SAM was determined to be approximately **$750,000** based on the estimated number of institutions (500) and average spending ($5,000).

- **Market Share Capture**: Based on user feedback and insights

In [ ]:
COMPILE_PROMPT = """"Present a comprehensive executive summary of the key findings
from the market analysis. This should include:
1. TAM, SAM, and SOM figures with detailed explanations of calculations and
assumptions
2. Top 5 market insights, including supporting data and implications for [Company
Name]
3. Detailed analysis of 5 main competitive advantages for [Company Name]
4. In-depth discussion of 5 primary market risks or challenges, including potential
mitigation strategies
5. Top 10 strategic recommendations, with rationale and expected outcomes for each
6. Projected market share after years 1, 3, and 5, including best-case, worst-case, and
most likely scenarios
7. Key market trends and their potential impact on [Company Name]'s growth trajectory
8. Analysis of customer segments and their evolving needs
9. Overview of the competitive landscape, including potential disruptors
10. Summary of go-to-market strategy, including key milestones and success metrics
This executive summary should be at least 5 pages long, providing a comprehensive
overview of the market analysis while remaining concise and actionable. Include a section
on methodology that explains the source verification process and any significant decisions
made regarding conflicting information."""

send_message(COMPILE_PROMPT)
last_message = get_last_message( )
print(last_message)

# Executive Summary of Market Analysis for EduConnect

## Introduction
This executive summary presents key findings from a comprehensive market analysis conducted for EduConnect, an innovative platform aimed at enhancing document verification processes for educational institutions in Ethiopia. The insights derived from this analysis pinpoint significant opportunities, competitive advantages, and implications for strategic growth.

## 1. TAM, SAM, and SOM Figures
**Total Addressable Market (TAM)**: The TAM for EduConnect is calculated at approximately **$2,500,000** annually. This was derived from estimating the **number of educational institutions** in Ethiopia (assumed at 500) and their average annual expenditure on document verification services (approximately $5,000 per institution).

**Serviceable Addressable Market (SAM)**: The SAM is assessed at **$750,000**, which reflects the estimated portion of the TAM likely to be served by EduConnect. This calculation assumes that **30%** o